# __APS DATA ANALYSER__
(Author removed for examination)
***

This is a script to analyse APS data and find the HOMO level for semiconductor materials. It was developed to analyse data from a KP Technology APS02 machine.
***

### 0. Preamble:

In [ ]:
#Import required modules, installing if not already installed
try:
  import pandas as pd
except:
  !pip install pandas

try:
  import matplotlib.pyplot as plt
except:
  !pip install matplotlib

try:
  from scipy import stats
except:
  !pip install scipy

try:
  from pathlib import Path
except:
  !pip install pathlib

try:
  from mpl_point_clicker import clicker #This is the module that allows us to select points on the graph using a cursor
except:
  !pip install mpl-point-clicker 


### 1. Import CSV data file
    Ensure file is in CSV format. 'Copy as path' and paste here.

In [ ]:
Data_File = input("Copy file path: ").strip('\"') #Ask users to input the file path. 
                                                  #Strip the "" that usually come with it. 

### 2. Set linear/baseline limits
    Click the key to switch between linear and baseline markers. Right-click to remove points. 
    
    Linear limits: Place two points to mark the rough area that contains a linear increase in photoemission current vs energy. The programme will find the best linear section within this energy range. 
    
    Baseline limits: Place two points to mark the energy range where the photoemission is roughly constant.

In [ ]:
%matplotlib nbagg 
#Allows interactive   figures

df = pd.read_csv(Data_File) #Read the CSV file and store as a pandas DataFrame
Energy, PE = df[' Energy (eV) '].dropna(), df[' CUBE RT(PE) '].dropna() #Extract relevant columns, remove null values
if len(Energy) != len(PE):
 print("Variables different sizes!") #Make sure the variables are the same length
                                     #If they are not the user can double check their data

fig, ax = plt.subplots(constrained_layout=True, figsize = (9,5)) #Ready the figure, set size
ax.plot(Energy, PE) #Plot the raw data
ax.set_title(Path(Data_File).stem) #Set the title from the name of the data file
ax.set_xlabel('Energy (eV)')
ax.set_ylabel('Photoemission current [1/3] (AU)') #Label axes

klicker = clicker(ax, ["Linear limits", "Baseline limits"], markers=["o", "x"]) #Marker parameters

### 3. Find HOMO
    The graph below will show the intercept of the linear fit and the baseline, where the HOMO level is found.
    Download the plot using the Save icon below.

In [ ]:
pos = klicker.get_positions() #Take the positions of the placed markers
pos_lin, pos_base = pos['Linear limits'].tolist(), pos['Baseline limits'].tolist() #Store the positions for the 
                                                                                   #two types of selected points, as lists
lin_x1, lin_x2 = pos_lin[0][0], pos_lin[1][0] #Take the x values of the first two linear points placed
lin_x_up = max(lin_x1, lin_x2)
lin_x_low = min(lin_x1, lin_x2) #Find the higher/lower of the two x values

base_x1, base_x2 = pos_base[0][0], pos_base[1][0] #Repeat for the baseline points
base_x_up = max(base_x1, base_x2)
base_x_low = min(base_x1, base_x2)

lin_df = df[df[' Energy (eV) '] > lin_x_low] 
lin1_df = lin_df[lin_df[' Energy (eV) '] < lin_x_up] #Take only the part of the dataframe within the range selected

base_df = df[df[' Energy (eV) '] > base_x_low]
base1_df = base_df[base_df[' Energy (eV) '] < base_x_up] #Repeat for the baseline section

base_en  = base1_df[' Energy (eV) ']
base_pe = base1_df[' CUBE RT(PE) '] #Get the relevant variables in the baseline section

#########################################################################################################################

#This part automatically finds the most linear set of consecutive points in the selected range

lin_len = 9 #This sets the length of the best linear section, 9 is default as some data files have a short linear section
             #The number of points selected will be lin_len +1

x_range = list(range(0, len(lin1_df) - lin_len)) #Create a list from 0 to (N - lin_len), for the algorithm to run through 
r_df = pd.DataFrame({'Range': [],'R': []}) #Create an empty dataframe to store the R values for each set of points

for n in x_range: #Pass each value in the range
    x_df = lin1_df.reset_index(drop = True).loc[n:lin_len+n] #Create a dataframe from the original that contains
                                                             #lin_len points, starting from point n.
                                                             #We drop index to allow us to go from 0; lin1_df would
                                                             #otherwise keep its indexing from the original df.
    x_en = x_df[' Energy (eV) '] 
    x_pe = x_df[' CUBE RT(PE) '] #Get relevant variables for this range
    
    a0, b0, r0_value, p0_value, std_err0 = stats.linregress(x_en, x_pe) #Calculate parameters of a linear fit in this range

    new_row = pd.DataFrame({'Range': [n],'R': [r0_value]}) #Create a new row which stores the starting n position 
                                                           #and the R value
    r_df = pd.concat([r_df, new_row ], ignore_index=True) #Add this to the dataframe. n value should equal the index

    #Pass through this algorithm until all possible sets of consecutive points have been measured

best_n = r_df['R'].idxmax() #Find the index of the highest R value. This is the starting point for the best section.
bestlin_df = lin1_df.reset_index(drop = True).loc[best_n:lin_len+best_n] #Store the best linear section as a dataframe

b_en = bestlin_df[' Energy (eV) ']
b_pe = bestlin_df[' CUBE RT(PE) '] #Get the variables for the best linear section
   
    
lin = stats.linregress(b_en, b_pe) #Calculate parameters of a linear fit in this range.

a = lin.slope #Gradient
b = lin.intercept #y-intercept
r_value = lin.rvalue #R value
a_err = lin.stderr #Standard error on the gradient
b_err = lin.intercept_stderr #Standard error on the y-intercept


#########################################################################################################################

#This part measures the baseline and calculates the HOMO. The different parts are then all plotted below. 
          
y0 = stats.trim_mean(base_pe, 0) #Take the mean of the y-values in the baseline range
base_err = stats.sem(base_pe) #Calculate the standard error on this mean

HM = (y0 - b)/a #Calculate the HOMO as the x-value where the linear fit intersects the baseline

HM_err = HM*(((base_err**2+b_err**2)/((y0 - b)**2))+(a_err**2/a**2))**(1/2) #Calculate the standard error on this 
                                                                            #HOMO level by adding errors in quadrature 

print("HOMO level = -", '{:.3f}'.format(round(HM, 3)) , "±", 
      '{:.3f}'.format(round(HM_err, 3)), "eV") #Print the HOMO and its error, rounded to 3 dec. places


lin_line_range = list(b_en) + [HM] #Create a list of best linear points to prepare for plotting the linear line,
                                   #add HM to extend this to the HOMO point
lin_line = [(x * a) + b - y0 for x in lin_line_range] #Create the linear fit, extended to the HOMO point and 
                                                      #intersecting the baseline at y=0


plt.figure(figsize=(9,5)) #Set the size of the figure
plt.plot(Energy, PE - y0) #Plot the raw data, shifted down so the baseline is at y=0
plt.plot(Energy, [0]*len(Energy), color = 'g') #Plot baseline across the whole data, at y=0
plt.plot(lin_line_range, lin_line, color = 'r') #Plot the linear line extending to the intersection
plt.scatter(b_en, (b_pe - y0), color = 'b', marker='.') #Show the points we used for the best linear section

plt.plot([HM, HM], [0, PE.mean()], ls='--', color = 'k', label = 'HOMO = -{:.3f} ± {:.3f} eV'
         .format(round(HM, 3),round(HM_err, 3))) #Label the intersection point with dashed line and add label for legend

plt.fill_betweenx([0, PE.mean()-1], [HM-HM_err, HM-HM_err], [HM+HM_err, HM+HM_err], 
                  facecolor = 'none', alpha=0.15, hatch = "\\\\", edgecolor="k") #Show error on HOMO using a filled range

plt.text(HM-HM_err, PE.mean(), '{:.3f}'.format(round(HM, 3)), fontsize = 28, ha = 'right', va = 'bottom') #Label the 
                                                                                    #HOMO point next to the dashed line
plt.text(HM-HM_err, PE.mean(), '± {:.3f} eV'.format(round(HM_err, 3)), fontsize = 11, ha = 'right', va = 'top') #Add the 
                                                                                    #HOMO errors below the HOMO label

plt.text(0.5*(pos_lin[0][0] + pos_lin[1][0]), 0.9*(PE.max()-y0), 'Linear range', 
         size = 10, ha = 'center', va = 'center', color = 'darkorange',alpha=0.6) #Labels the user-chosen linear range
plt.text(0.5*(pos_base[0][0] + pos_base[1][0]), 0.9*(PE.max()-y0), 'Baseline range', 
         size = 10, ha = 'center', va = 'center', color = 'g',alpha=0.6) #Labels the user-chosen baseline range

for i in [0,1]:
    plt.axvline(x=pos_lin[i][0], color = 'orange',ls=':',alpha=0.25)
    plt.axvline(x=pos_base[i][0], color = 'g',ls=':',alpha=0.25) #This shows the user-chosen points for the linear and 
                                                                 #baseline limits
    
#plt.title(Path(Data_File).stem) #Label the figure with a title from the data file
plt.title('Example')
plt.xlabel('Energy (eV)') #Label axes
plt.ylabel('Photoemission current [1/3] (AU)') #Our photoemission current has arbitrary units
plt.legend(loc = 'upper left') #Show legend in upper right corner

#### Appendix
    Here we show the 'linear' and 'baseline' sections individually, with their specific parameters and errors

In [ ]:
#Linear range
plt.figure() #New figure
#plt.title('{} - Linear range'.format(Path(Data_File).stem)) #Set title
li_en = lin1_df[' Energy (eV) '] 
li_pe = lin1_df[' CUBE RT(PE) '] #Get relevant variables for the linear section
plt.plot(li_en, li_pe) #Plot our variables
plt.scatter(b_en, (b_pe), color = 'b', marker='.') #Show the best linear points found by the algorithm
plt.plot(b_en, a*b_en + b, 
         label = 'Gradient = {:.3f} ± {:.3f}\ny-intercept = {:.3f} ± {:.3f}\nR\u00b2 value = {:.4f}\nn = {}'
         .format(round(a,3),round(a_err,3),round(b,3), round(b_err,3), round(r_value**2,4), len(b_en))
         , color = 'r') #Plot the fitted line with appropriate parameters labelled for the legend

for i in [0,1]:
    plt.axvline(x=pos_lin[i][0], color = 'orange',ls=':',alpha=1) #Show the limits of the user-chosen range
    
plt.xlabel('Energy (eV)') #Label axes
plt.ylabel('Photoemission current [1/3] (AU)') 
plt.legend(loc = 'lower right') #Show legend in lower right corner


plt.title('Linear range')

In [ ]:
sdb = stats.tstd(base_pe) #Calculate the standard deviation of the baseline data

plt.figure() #New figure
#plt.title('{} - Baseline range'.format(Path(Data_File).stem)) #Set title
plt.plot(base_en, base_pe) #Plot variables in the baseline range

plt.plot(base_en, [y0]*len(base_en), color = 'g', 
         label = 'Mean = {:.3f} ± {:.3f}\nStandard deviation = {:.4f}\nn={}'
         .format(round(y0,3),round(base_err,3),round(sdb,4),len(base_en))) #Plot the baseline, with mean and stan. dev. values labelled

plt.fill_between(base_en, [y0+base_err]*len(base_en), [y0-base_err]*len(base_en),
                 facecolor = 'teal', alpha=0.25) #Show the error on the mean with a filled range

for i in [0,1]:
        plt.axvline(x=pos_base[i][0], color = 'g',ls=':',alpha=1) #Show the limits of the user-chosen range
        
plt.xlabel('Energy (eV)') #Label axes
plt.ylabel('Photoemission current [1/3] (AU)') #Our photoemission current has arbitrary units
plt.legend(loc = 'lower right') #Show legend in the lower right corner

plt.title('Baseline range')